In [ ]:
!nvidia-smi

In [ ]:
import os
import pandas as pd
import numpy as np
import shutil
import sys
import tqdm.notebook as tq
from collections import defaultdict

import torch
import torch.nn as nn

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


In [ ]:
df_data = pd.read_csv('multi_label_binarizer_MEISD.csv')

In [ ]:
df_data.head()

In [ ]:
# For the multilabel classification we use:
columns = ['Utterances', 'sentiment_0', 'sentiment_1', 'sentiment_2']
multi_columns = df_data[columns].copy()

In [ ]:
multi_columns

In [ ]:
df_data['label'] = multi_columns[['sentiment_0', 'sentiment_1', 'sentiment_2']].idxmax(axis=1)
df_data['label'] = df_data['label'].apply(lambda x: int(x.split('_')[1]))
df_data = df_data[['Utterances', 'label']]
df_data

In [ ]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# Test the tokenizer
test_text = "We are testing BERT tokenizer."
# generate encodings
encodings = tokenizer.encode_plus(test_text,
                                  add_special_tokens = True,
                                  max_length = 50,
                                  truncation = True,
                                  padding = "max_length",
                                  return_attention_mask = True,
                                  return_tensors = "pt")
# we get a dictionary with three keys (see: https://huggingface.co/transformers/glossary.html) 
encodings

In [ ]:
token_lens = []

for txt in df_data['Utterances']:
    tokens = tokenizer.encode(txt, max_length=512)
    token_lens.append(len(tokens))


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.distplot(token_lens)
plt.xlim([0, 40])
plt.xlabel('Token count')


In [ ]:
# Hyperparameters
MAX_LEN = 50 #128  # wiekszosc tokenow zdaje sie byc ponizej 40, klasycznie wklada sie tu 256, my przystaniemy na 50
TRAIN_BATCH_SIZE = 8 #16 #32 
#Czasami, przy bardzo niskim tempie uczenia i zbyt dużych batchach, model może wolniej konwergować. Spróbuj zmniejszyć wielkość batcha, np. z 16 do 8.
VALID_BATCH_SIZE = 8 #16 #32
TEST_BATCH_SIZE = 8 #16 #32
EPOCHS = 10
LEARNING_RATE = 2e-05
# Ustawienie bardzo niskiego współczynnika uczenia (np. 1e-05) może spowodować, że model uczy się bardzo wolno, co prowadzi do sytuacji, w której po wielu epokach nie ma znaczącej poprawy w wynikach walidacji.

THRESHOLD = 0.5 # threshold for the sigmoid


In [ ]:
from sklearn.model_selection import train_test_split
# split into train and test
df_train, df_test = train_test_split(df_data, random_state=77, test_size=0.30, shuffle=True)
# split test into test and validation datasets
df_test, df_valid = train_test_split(df_test, random_state=88, test_size=0.50, shuffle=True)

In [ ]:
columns = multi_columns.columns
categor_freq = multi_columns[columns[1:]].sum() / multi_columns.shape[0]
categor_freq

In [ ]:
class_distribution = multi_columns[['sentiment_0', 'sentiment_1', 'sentiment_2']].sum()
print(class_distribution)

In [ ]:
import matplotlib.pyplot as plt

# Wykres rozkładu klas
class_distribution.plot(kind='bar')
plt.title('Class Distribution')
plt.xlabel('Sentiment Class')
plt.ylabel('Number of Samples')
plt.xticks(rotation=0)
plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (15, 3)
plt.bar(categor_freq.index, categor_freq.values)
_ = plt.xticks(rotation=45)

In [ ]:
print(f"Train: {df_train.shape}, Test: {df_test.shape}, Valid: {df_valid.shape}")

In [ ]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, df, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.df = df
        self.utterances = list(df['Utterances'])
        # Upewnij się, że etykiety są typu całkowitego (int)
        self.targets = self.df['label'].astype(int).values  # Zapewnij typ int
        self.max_len = max_len

    def __len__(self):
        return len(self.utterances)

    def __getitem__(self, index):
        utterances = str(self.utterances[index])  # 'index' jest prawidłowe
        utterances = " ".join(utterances.split())  # Usuwa niepotrzebne białe znaki

        inputs = self.tokenizer.encode_plus(
            utterances,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        target = torch.tensor(self.targets[index], dtype=torch.long)  # Zapewnij typ long
        # print(f"Target dtype: {target.dtype}")  # Debugging

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs["token_type_ids"].flatten(),
            'targets': torch.tensor(self.targets[index], dtype=torch.long),  # Zapewnij typ long
            'utterances': utterances
        }


In [ ]:
target_list = list(df_data.columns)
target_list = target_list[1:]
target_list

In [ ]:
train_dataset = CustomDataset(df_train, tokenizer, MAX_LEN)
valid_dataset = CustomDataset(df_valid, tokenizer, MAX_LEN)
test_dataset = CustomDataset(df_test, tokenizer, MAX_LEN)

In [ ]:
data = next(iter(train_dataset))

In [ ]:
print(data.keys())

print(data['input_ids'].shape)
print(data['attention_mask'].shape)
print(data['targets'].shape)

In [ ]:
# Data loaders
train_data_loader = torch.utils.data.DataLoader(train_dataset,
                                                batch_size=TRAIN_BATCH_SIZE,
                                                shuffle=True,
                                                num_workers=0
                                                )

val_data_loader = torch.utils.data.DataLoader(valid_dataset,
                                              batch_size=VALID_BATCH_SIZE,
                                              shuffle=False,
                                              num_workers=0
                                              )

test_data_loader = torch.utils.data.DataLoader(test_dataset,
                                               batch_size=TEST_BATCH_SIZE,
                                               shuffle=False,
                                               num_workers=0
                                               )

In [ ]:
class BERTSentimentClass(torch.nn.Module):
    def __init__(self):
        super(BERTSentimentClass, self).__init__()
        self.bert_model = BertModel.from_pretrained('bert-base-uncased', return_dict=True)
        self.dropout = torch.nn.Dropout(0.5)
        self.linear = torch.nn.Linear(768, 3)
        #self.softmax = nn.Softmax(dim=1) #remove for sentiment analysis
        #CrossEntropyLoss automatycznie aplikuje funkcję softmax, więc nie ma potrzeby używać Softmax w modelu.


    def forward(self, input_ids, attn_mask, token_type_ids):
        output = self.bert_model(
            input_ids,
            attention_mask=attn_mask,
            token_type_ids=token_type_ids
        )
        #pooler_output = self.pooler_output
        dropout_output = self.dropout(output.pooler_output)
        linear_output = self.linear(dropout_output)
        # output = self.softmax(linear_output)
        return linear_output

model = BERTSentimentClass()

# # Freezing BERT layers:
# for param in model.bert_model.parameters():
#     param.requires_grad = False

model.to(device)


In [ ]:
class_distribution = multi_columns[['sentiment_0', 'sentiment_1', 'sentiment_2']].sum()
total_samples = sum(class_distribution)
class_weights = [total_samples / count for count in class_distribution]
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)
class_weights

In [ ]:
def loss_fn(outputs, targets):
    #print(f"Outputs dtype: {outputs.dtype}")  # Debugging
    #print(f"Targets dtype: {targets.dtype}")  # Debugging

    return torch.nn.CrossEntropyLoss(weight=class_weights)(outputs, targets)
#change for sentiment analysis

In [ ]:
# TensorBoard writer
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter(log_dir='logs')

# Harmonogram zmiany learning rate
from torch.optim.lr_scheduler import StepLR

In [ ]:
from transformers import AdamW

# define the optimizer
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)

In [ ]:
# Training of the model for one epoch
def train_model(training_loader, model, optimizer):
    losses = []
    correct_predictions = 0
    num_samples = 0
    # set model to training mode (activate droput, batch norm)
    model.train()
    # initialize the progress bar
    loop = tq.tqdm(enumerate(training_loader), total=len(training_loader),
                   leave=True, colour='steelblue')
    for batch_idx, data in loop:
        ids = data['input_ids'].to(device, dtype=torch.long)
        mask = data['attention_mask'].to(device, dtype=torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
        targets = data['targets'].to(device, dtype=torch.long)


        # forward
        outputs = model(ids, mask, token_type_ids)  # (batch,predict)=(32,8)
        loss = loss_fn(outputs, targets)
        losses.append(loss.item())


        # Debugging
        print(f"Batch {batch_idx}:")
        print(f"Input IDs shape: {ids.shape}")
        print(f"Targets shape: {targets.shape}")
        print(f"Outputs shape: {outputs.shape}")
        print(f"Loss: {loss.item()}")


        # training accuracy, apply sigmoid, round (apply thresh 0.5)
        # change for sentiment analysis becuase we have switch to Cross Entropy Loss
        outputs = torch.argmax(outputs, axis=1).cpu().detach()
        targets = targets.cpu().detach().numpy()
        correct_predictions += np.sum(outputs == targets)
        num_samples += targets.size  # total number of elements in the 2D array

        # Debugging
        #print(f"Raw outputs: {outputs}")
        #print(f"Targets: {targets}")
        

        # backward
        optimizer.zero_grad()
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        # grad descent step
        optimizer.step()
        
        # Debugging
        #for name, param in model.named_parameters():
        #    if param.requires_grad:
        #        print(f"Gradients for {name}: {param.grad}")

        # Log loss and accuracy to TensorBoard
        writer.add_scalar('Loss/train', loss.item(), epoch * len(training_loader) + batch_idx)
        writer.add_scalar('Accuracy/train', correct_predictions / num_samples, epoch * len(training_loader) + batch_idx)


        # Update progress bar
        loop.set_description(f"Epoch {epoch}")
        loop.set_postfix(batch_loss=loss.item())

    # returning: trained model, model accuracy, mean loss
    return model, float(correct_predictions) / num_samples, np.mean(losses)


In [ ]:
def eval_model(validation_loader, model, optimizer):
    losses = []
    correct_predictions = 0
    num_samples = 0
    # set model to eval mode (turn off dropout, fix batch norm)
    model.eval()
    all_preds = []
    all_labels = []


    with torch.no_grad():
        for batch_idx, data in enumerate(validation_loader, 0):
            ids = data['input_ids'].to(device, dtype = torch.long)
            mask = data['attention_mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.long)
            outputs = model(ids, mask, token_type_ids)

            loss = loss_fn(outputs, targets)
            losses.append(loss.item())

            # validation accuracy
            # add sigmoid, for the training sigmoid is in BCEWithLogitsLoss
            # change for sentiment analysis becuase we have switch to Cross Entropy Loss
            outputs = torch.argmax(outputs, axis=1).cpu().detach()      
            targets = targets.cpu().detach().numpy()
            correct_predictions += np.sum(outputs==targets)
            num_samples += targets.size   # total number of elements in the 2D array
            all_preds.extend(outputs.numpy())
            all_labels.extend(targets)


# Log validation loss and accuracy to TensorBoard
    writer.add_scalar('Loss/validation', np.mean(losses), epoch)
    writer.add_scalar('Accuracy/validation', float(correct_predictions) / num_samples, epoch)

    # Confusion matrix
    # cm = confusion_matrix(all_labels, all_preds)
    # plot_confusion_matrix(cm, class_names=['class0', 'class1', 'class2'], epoch=epoch)


    return float(correct_predictions)/num_samples, np.mean(losses)


In [ ]:
from collections import defaultdict
import torch
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import io

history = defaultdict(list)
best_accuracy = 0
writer = SummaryWriter(log_dir='logs')

def plot_confusion_matrix(cm, class_names, epoch):
    figure = plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.title(f'Confusion Matrix at Epoch {epoch}')

    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    image = torch.tensor(np.frombuffer(buf.getvalue(), dtype=np.uint8)).float()
    writer.add_image('Confusion Matrix', image, epoch)

    plt.close(figure)  


In [ ]:
from torch.utils.tensorboard import SummaryWriter

In [ ]:
for epoch in range(1, EPOCHS + 1):
    print(f'Epoch {epoch}/{EPOCHS}')
    model, train_acc, train_loss = train_model(train_data_loader, model, optimizer)
    val_acc, val_loss = eval_model(val_data_loader, model, optimizer)

    # Logowanie strat i dokładności do TensorBoard
    writer.add_scalar('Loss/train', train_loss, epoch)
    writer.add_scalar('Accuracy/train', train_acc, epoch)
    writer.add_scalar('Loss/validation', val_loss, epoch)
    writer.add_scalar('Accuracy/validation', val_acc, epoch)

    print(f'train_loss={train_loss:.4f}, val_loss={val_loss:.4f}, train_acc={train_acc:.4f}, val_acc={val_acc:.4f}')

    history['train_acc'].append(train_acc)
    history['train_loss'].append(train_loss)
    history['val_acc'].append(val_acc)
    history['val_loss'].append(val_loss)

    if val_acc > best_accuracy:
        torch.save(model.state_dict(), "best_model_state.bin")
        best_accuracy = val_acc

    all_preds = []  
    all_labels = [] 

    with torch.no_grad():
        for data in val_data_loader:
            ids = data['input_ids'].to(device, dtype=torch.long)
            mask = data['attention_mask'].to(device, dtype=torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype=torch.long)

            outputs = model(ids, mask, token_type_ids)
            preds = torch.argmax(outputs, axis=1).cpu().detach().numpy() 
            labels = targets.cpu().detach().numpy() 

            all_preds.extend(preds)
            all_labels.extend(labels)

    # Oblicz confusion matrix
    # cm = confusion_matrix(all_labels, all_preds)
    # plot_confusion_matrix(cm, class_names=['class0', 'class1', 'class2'], epoch=epoch)

writer.close()


# 1. Overfitting:
Opis: Niski train_loss (0.2796) wskazuje na to, że model dobrze dopasowuje się do danych treningowych, ale wysoki val_loss (2.9933) oraz brak poprawy w val_acc (0.0000) sugerują, że model może się przeuczać, czyli dopasowuje się zbyt mocno do danych treningowych i traci zdolność do generalizacji na danych testowych.
Rozwiązanie:
Dodanie technik regularyzacyjnych, jak dropout, L2 regularization.
Wykorzystanie większego zbioru danych.
Zastosowanie wcześniejszego zatrzymania (early stopping), aby przerwać trening, gdy model zaczyna się przeuczać.
# 2. Zbyt niski learning rate:
Opis: Ustawienie bardzo niskiego współczynnika uczenia (np. 1e-05) może spowodować, że model uczy się bardzo wolno, co prowadzi do sytuacji, w której po wielu epokach nie ma znaczącej poprawy w wynikach walidacji.
Rozwiązanie: Spróbuj zwiększyć learning rate np. do 1e-04 i zobacz, czy poprawia to wyniki. Zbyt niski współczynnik uczenia może blokować osiąganie optymalnych wyników.
# 3. Zbyt skomplikowany model:
Opis: Jeśli model jest zbyt złożony w stosunku do dostępnych danych, może to prowadzić do overfittingu. Model nauczy się bardzo dobrze danych treningowych, ale nie będzie w stanie dobrze generalizować.
Rozwiązanie: Możesz spróbować uprościć model (np. mniejsza liczba warstw, mniejsza liczba neuronów) lub zebrać większy zbiór danych, jeśli to możliwe.
# 4. Problemy z danymi:
Opis: Dane walidacyjne mogą zawierać problemy, takie jak błędnie oznaczone próbki, brak różnorodności, lub mogą nie być reprezentatywne dla danych treningowych.
Rozwiązanie: Sprawdź, czy dane walidacyjne są dobrze zrównoważone i poprawnie oznaczone. Ewentualnie przetestuj na innym zbiorze walidacyjnym.
# 5. Złe inicjalizacje wag lub problemy z optymalizacją:
Opis: Wysoki val_loss i brak poprawy w val_acc mogą wskazywać na problemy z optymalizacją. Np. złe inicjalizacje wag lub nieodpowiedni optymalizator mogą powodować, że model nie znajduje optymalnych rozwiązań.
Rozwiązanie: Spróbuj zmienić optymalizator (np. Adam na RMSprop), lub zastosować inne techniki inicjalizacji wag.
# 6. Zbyt zróżnicowane klasy:
Opis: Jeśli Twoje klasy są bardzo niezrównoważone, to model może mieć problem z nauczeniem się klasyfikacji rzadkich klas.
Rozwiązanie: Upewnij się, że klasy są zrównoważone lub użyj metod radzenia sobie z niezrównoważonymi danymi (np. class weights w funkcji straty).


In [ ]:
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (10, 7)
plt.plot(history['train_acc'], label='train accuracy')
plt.plot(history['val_acc'], label='validation accuracy')
plt.title('Training history')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.ylim([0, 1])
plt.grid()


In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

def plot_confusion_matrix(cm, class_names):
    fig, ax = plt.subplots(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", ax=ax)
    ax.set_xlabel("Predicted labels")
    ax.set_ylabel("True labels")
    ax.set_title("Confusion Matrix")
    ax.set_xticklabels(class_names)
    ax.set_yticklabels(class_names)
    return fig

# Tworzenie confusion matrix po ewaluacji
cm = confusion_matrix(all_labels, all_preds)

# Logowanie confusion matrix jako obraz
fig = plot_confusion_matrix(cm, class_names=['class0', 'class1', 'class2'])
writer.add_figure('Confusion matrix', fig, epoch)


In [ ]:
fig